![MuJoCo banner](https://raw.githubusercontent.com/google-deepmind/mujoco/main/banner.png)

# <h1><center>Tutorial  <a href="https://colab.research.google.com/github/google-deepmind/mujoco_warp/blob/main/notebooks/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

This notebook provides an introductory tutorial for [**MuJoCo Warp (MJWarp)**](https://github.com/google-deepmind/mujoco_warp/), an implementation of MuJoCo written with [NVIDIA Warp](https://github.com/NVIDIA/warp).

**A Colab runtime with GPU acceleration is required.** If you're using a CPU-only runtime, you can switch using the menu "Runtime > Change runtime type".










# All imports

In [ ]:
!uv pip install "git+https://github.com/google-deepmind/mujoco_warp.git"
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!uv pip install mediapy

# Set up GPU rendering.
from google.colab import files

import distutils.util
import os
import subprocess
import mediapy as media
import warp as wp
import numpy as np
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# avoid warp output in cells
wp.config.quiet = True

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

try:
  print('Checking that the installation succeeded:')
  import mujoco
  import mujoco_warp as mjw
  mjw.put_model(mujoco.MjModel.from_xml_string('<mujoco/>'))
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# Introduction to MuJoCo Warp

MuJoCo Warp (MJWarp) is an implementation of MuJoCo using [NVIDIA Warp](https://github.com/NVIDIA/warp).  MJWarp is optimized for running large batches of simulation in parallel on NVIDIA GPUs.  You can use MJWarp to rapidly train or evaluate robot models with faithful transfer between the simulated robot and the real robot.

In this notebook, we will demonstrate the basic usage of MJWarp. Let's get started with a simpler example! The entrypoint into MJWarp is through MuJoCo, so first we load a MuJoCo model:

In [ ]:
xml = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <body name="box_and_sphere" euler="0 0 -30">
      <joint name="swing" type="hinge" axis="1 -1 0" pos="-.2 -.2 -.2"/>
      <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
      <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
    </body>
  </worldbody>
</mujoco>
"""

mj_model = mujoco.MjModel.from_xml_string(xml)


Let's run the simulation in MuJoCo and render the trajectory. This example is taken from the [MuJoCo tutorial](https://colab.sandbox.google.com/github/google-deepmind/mujoco/blob/main/python/tutorial.ipynb).

In [ ]:
mj_data = mujoco.MjData(mj_model)
renderer = mujoco.Renderer(mj_model)

# enable joint visualization option:
scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True

duration = 3.8  # (seconds)
framerate = 60  # (Hz)

frames = []
mujoco.mj_resetData(mj_model, mj_data)
while mj_data.time < duration:
  mujoco.mj_step(mj_model, mj_data)
  if len(frames) < mj_data.time * framerate:
    renderer.update_scene(mj_data, scene_option=scene_option)
    pixels = renderer.render()
    frames.append(pixels)

# Simulate and display video.
media.show_video(frames, fps=framerate)

Next we take the MuJoCo `MjModel` and `MjData`, and place them on the GPU using MJWarp.

In [ ]:
model = mjw.put_model(mj_model)
data = mjw.put_data(mj_model, mj_data)

Below, we print the `qpos` from MuJoCo and MJWarp. Notice that the `qpos` for the `mjData` is a numpy array living on the CPU, while the `qpos` for `Data` is a Warp array living on the GPU device.

In [ ]:
print(mj_data.qpos, type(mj_data.qpos))
print(data.qpos, type(data.qpos), data.qpos.device)

Now let's run the same exact simulation on the GPU device using MJWarp!

In the example below, we use `mjw.step` instead of `mujoco.mj_step`. For each frame, we convert the `mjw.Data` back to `mjData` so that we can use the MuJoCo renderer.


In [ ]:
mjw.reset_data(model, data)
frames = []
while data.time.numpy() < duration:
  mjw.step(model, data)
  if len(frames) < data.time.numpy() * framerate:
    mjw.get_data_into(mj_data, mj_model, data)
    renderer.update_scene(mj_data, scene_option=scene_option)
    frames.append(renderer.render())

media.show_video(frames, fps=framerate)

We will improve the performance of the simulation in two ways:

1) Create many copies of the simulation state and step them in parallel
2) Record the sequence of CUDA operations in `mjw.step`, so that they can be [executed on GPU as a single graph](https://nvidia.github.io/warp/modules/concurrency.html#synchronization-and-graph-capture).

In the example below, we create 4096 copies of `Data` on GPU, and we execute a single graph on device using Warp's `ScopedCapture` feature:

In [ ]:
data = mjw.put_data(mj_model, mj_data, nworld=4096)

with wp.ScopedCapture() as capture:
  mjw.step(model, data)

wp.capture_launch(capture.graph)

MJWarp not only operates on batches of `Data` - it also operate on batches of `Model` fields.

When MJWarp finds a batched `Model` field, it distributes field values evenly across the `Data` copies.  For example, the following steps 4096 copies of `Data`, and half of them have upside down gravity.

In [ ]:
data = mjw.put_data(mj_model, mj_data, nworld=4096)
model.opt.gravity = wp.array([[0, 0, -9.81], [0, 0, 9.81]], dtype=wp.vec3f)
mjw.step(model, data)

# MuJoCo Warp for robot learning

Although you can use MJWarp directly, it is also integrated into some existing toolkits for training robots using machine learning:

* [MuJoCo Playground](https://github.com/google-deepmind/mujoco_playground) is
a comprehensive suite of GPU-accelerated environments for robot learning research and sim-to-real.  It is written in JAX but integrates MJWarp for physics.

* [Isaac Lab](https://github.com/isaac-sim/IsaacLab/tree/feature/newton) is a GPU-accelerated, open-source framework designed by NVIDIA for robotics research.  It has beta support for MJWarp through a new physics platform called[Newton](https://developer.nvidia.com/newton-physics).

* [mjlab](https://github.com/mujocolab/mjlab) combines Isaac Lab's proven API with best-in-class MuJoCo physics to provide lightweight, modular abstractions for RL robotics research and sim-to-real deployment.

👋 Tawa pona!